In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt

# inputs

In [29]:
qw = 5000
rw = 0.25
re = 1000
pinit = 3000
perm = 50
poro = 0.2
height = 100
ct = 1e-5
N = 4
delta_time = 1
visc = 5

In [25]:
def gamma(rw,re,N):
    return (re/rw)**(1/N)

In [26]:
def r(rw,gamma,N):
    r1 = (rw*np.log(gamma)/(1-1/gamma)).tolist()
    return [r1*gamma**i for i in range(-1,N+1)]

In [34]:
def flow(visc,perm,deltaz,deltat,gamma):
    return (perm/visc*deltaz*deltat/np.log(gamma)).tolist()

In [43]:
def storage(poro,deltat,deltaz,delta_time,rvalues):
    gamma = rvalues[1]/rvalues[0]
    lhs = (poro*deltat*deltaz)/(2*delta_time)
    rvalues = np.asarray(rvalues)
    rhs = (rvalues[2:]**2-2*rvalues[1:-1]**2+rvalues[:-2]**2)/(2*np.log(gamma))
    return lhs*rhs

In [44]:
gm = gamma(rw,re,N)
rv = r(rw,gm,N)

In [48]:
t = flow(visc,perm,height,2*np.pi,gm)

In [49]:
a = storage(poro,2*np.pi,height,delta_time,rv)

In [75]:
T = np.zeros((N,N))
A = np.zeros((N,N))
Q = np.zeros((N,1))
P = np.ones((N,1))*pinit

In [76]:
for i in range(N):
    if i>0:
        T[i,i-1] = -t
        T[i,i] += t
    if i<N-1:
        T[i,i+1] = -t
        T[i,i] += t
        
A = np.diag(a*ct)

Q[0] = -5000

In [77]:
T = T*19.18/3030; T

array([[ 19.18135149, -19.18135149,   0.        ,   0.        ],
       [-19.18135149,  38.36270298, -19.18135149,   0.        ],
       [  0.        , -19.18135149,  38.36270298, -19.18135149],
       [  0.        ,   0.        , -19.18135149,  19.18135149]])

In [78]:
A

array([[3.26321208e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.06383653e-01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.30528483e+01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.25534612e+02]])

In [79]:
Q

array([[-5000.],
       [    0.],
       [    0.],
       [    0.]])

In [80]:
P

array([[3000.],
       [3000.],
       [3000.],
       [3000.]])

In [82]:
for _ in range(3):
    P = np.linalg.solve(T+A,np.dot(A,P)+Q)
    print(P)

[[2328.8016828 ]
 [2589.35734487]
 [2845.49465658]
 [2996.49157654]]
[[2257.19570754]
 [2517.85337479]
 [2777.74168797]
 [2991.52432284]]
[[2225.74703878]
 [2486.41153776]
 [2746.73773516]
 [2985.9658419 ]]
